# Hydrogenic computations

In [1]:
using JAC

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b5211b19a5aa56f2c76a4f66df0ec7b35773f4c6-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b5211b19a5aa56f2c76a4f66df0ec7b35773f4c6-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

Perhaps, the simplest atomic computations can be made for hydrogenic ions. If we consider, for example, hydrogen-like argon (Z=18), let us first determine and compare the energies of the 1s and 2p levels from the (non-relativistic) Schroedinger equation with those from Dirac's relativistic equation by

In [2]:
Z     = 36.0
e1s   = HydrogenicIon.energy(Shell("1s"), Z)           
e2p   = HydrogenicIon.energy(Shell("2p"), Z)
e2p_1 = HydrogenicIon.energy(Subshell("2p_1/2"), Z)   
e2p_3 = HydrogenicIon.energy(Subshell("2p_3/2"), Z)

  Energie for shell 1s is [in eV]: -6.48000000e+02
  Energie for shell 2p is [in eV]: -1.62000000e+02
  Energie for subshell 2p_1/2 is [in eV]: -1.65626275e+02
  Energie for subshell 2p_3/2 is [in eV]: -1.62704858e+02


-162.70485811496013

While the (one-electron) energies are displayed also in the default units [here eV] by the function  HydrogenicIon.energy(), all **computations are internally performed and returned always in atomic units** (here, for example, by the variables  e1s, e2p,  ...), if not stated otherwise in the description of some particular function. However, the units of energies, rates and several other physical properties can be quite easily 'converted' among each other, for instance, by:

In [3]:
e1s_eV = JAC.convert("energy: from atomic to eV", e1s)
e1s_au = JAC.convert("energy: from eV to atomic", e1s_eV)

-648.0

... and similarly also for other energy units as well as for other physical entities; cf. ? JAC.convert(). --- We here request and call the explicit form  **JAC.convert()** in order not to confuse this function with the Base.convert() function.


From these energies, we can easily compute the fine-structure splitting of the 2p level into the 2p_1/2 and 2p_3/2 fine-structure levels and compare them to the experimental value of about 53 eV

In [10]:
e13 = e2p_1 - e2p_3
e13_eV = JAC.convert("energy: from atomic to eV", e13)

-79.4958080164456

Apart from the (single-electron) energies, we can readily generate the radial orbitals, i.e.  P(r) from the non-relativistic theory or  P(r) and Q(r)  from the relativistic theory. However, the general solution from Dirac's equation are presently not (fully) implemented. Instead, a good approximation is obtained by just applying the **kinetic-balance condition** to the non-relativistic P^nonrel (r)  function and, then, by re-normalizing the large and small components (P and Q) together. 
        
The non-relativistic radial orbitals  P(r)  can obtained either for a single r-value,  for a list of r-values as well as for all r-values on a given grid. -- We here apply an exponential grid to make further use of these functions in computing various expectation values:

In [6]:
grid   = JAC.Radial.Grid("grid: exponential")
Pnr_1s = HydrogenicIon.radialOrbital(Shell("1s"), Z, grid)
Pnr_2p = HydrogenicIon.radialOrbital(Shell("2p"), Z, grid)

390-element Array{Float64,1}:
 0.0                   
 1.668994977215861e-11 
 7.022638420841005e-11 
 1.6628341383997606e-10
 3.1122409632155873e-10
 5.121776145391072e-10 
 7.771248126739855e-10 
 1.1149929814381725e-9 
 1.5357608780248208e-9 
 2.050575070595521e-9  
 2.671878812396818e-9  
 3.4135547780208284e-9 
 4.291083135176646e-9  
 ⋮                     
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   

All these radial orbitals can be plotted also by the standard plot procedures; within JAC, however, special plotting functions are provided only for (relativistic) radial Orbital(s) which carry all their information with them:

In [11]:
? JAC.Orbital

`struct  Orbital`  ... defines a type for a single-electron radial orbital function with a large and small component, and which can refer to          either the standard or an explicitly given grid due to the logical flag isStandardGrid. Bound-state orbitals with energy < 0 are           distinguished from free-electron orbitals by the flag isBound. – Note that the arrays P, Q and grid cannot be defined only by the           standard constructor but are typically set explicitly after an instance of this type has been created.

```
+ subshell        ::Subshell          ... Relativistic subshell.
+ isBound         ::Bool              ... Logical flag to distinguish between bound (true) and free-electron 
                                          orbitals (false).
+ useStandardGrid ::Bool              ... Logical flag for using the standard grid (true) or an explicitly 
                                          given grid (false).
+ energy          ::Float64           ... Single-electron energies of bound orbitals are always negative.
+ P               ::Array{Float64,1}  ... Large and ..
+ Q               ::Array{Float64,1}  ... small component of the radial orbital.
+ grid            ::Array{Float64,1}  ... explic. defined radial grid array for P, Q, if StandardGrid = false.
```

---

`JAC.Orbital(subshell::Subshell, energy::Float64)`  ... constructor for given subshell and energy, and where isStandardGrid is set to true;      the grid must be defined explicitly and the large and small components are not yet defined in this case.

---

`JAC.Orbital(label::String, energy::Float64)`  ... constructor for given string identifier and energy, and where isStandardGrid is set       to true; the grid must be defined explicitly and the large and small components are not yet defined in this case.


To display the non-relativistic  Pnr_1s(r)  orbitals, we need first to define a standard grid as well as such an (instance of the) Orbital by using its standard constructor, and by setting the small  Q(r)  component simply to zero:

In [13]:
JAC.define("standard grid", grid)
Qnr_1s   = zeros( length(Pnr_1s) )
nrOrb_1s = Orbital( Subshell("1s_1/2"), true, true, -162.0, Pnr_1s, Qnr_1s, Radial.Grid())

Re-define the standard grid with 390 grid points.


Bound-state orbital 1s_1/2 with energy -162.0 a.u. is defined with 390 (grid) points on the standard grid: 
Large component P: [0.0, 4.42981e-5, 9.0867e-5, 0.000139823, 0.000191289, 0.000245393, 0.00030227, 0.000362063, 0.000424921, 0.000491002  …  0.00096501, 0.00105877, 0.00115734, 0.00126096, 0.0013699, 0.00148441, 0.0016048, 0.00173135, 0.00186439, 0.00200425]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Small component Q: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Defined on Grid:   [0.0, 1.02542e-7, 2.10342e-7]  ...  [481.852, 506.557, 532.529, 559.832]


We can also compute a relativistic 1s_1/2 orbital on the given grid and plot the two 1s-orbitals together; while the large components of these orbitals coincide of course, due to its special construction, the small compoments differ:

In [14]:
rOrb_1s = HydrogenicIon.radialOrbital(Subshell("1s_1/2"), Z, grid)
JAC.plot("radial orbitals: large", Orbital[nrOrb_1s, rOrb_1s], grid; N = 230)
JAC.plot("radial orbitals: small", Orbital[nrOrb_1s, rOrb_1s], grid; N = 230) 

  Energie for subshell 1s_1/2 is [in eV]: -6.59583512e+02
JAC.HydrogenicIon.radialOrbital():  for subshell 1s_1/2 : norm-before = 1.0508017921952695, norm-after = 0.9999999999999999


UndefVarError: UndefVarError: plot not defined

We can use these orbitals to quickly evaluate some overlap integrals and <r^k> expectation values; for this, we make use of the non-relativistic 2p radial orbital Pnr_2p from above an compute the normalization  and <r> expectation values:

In [ ]:
N_1s = JAC.RadialIntegrals.overlap(Pnr_1s, Pnr_1s, grid)
N_2p = JAC.RadialIntegrals.overlap(Pnr_2p, Pnr_2p, grid)

For the sake of simplicity, all hydrogenic computations are performed for a point-like nucleus. For many-electron computations, in constrast, a more realistic nucleus can be taken into accout by selecting a proper nuclear model. See the tutorial: Setting the nuclues   or   **? Nuclear.Model**  ... for further details.

In [15]:
 ? Nuclear.Model

`struct  Nuclear.Model`  ... defines a type for the nuclear model, i.e. for its form and parameters.

```
+ Z        ::Float64         ... nuclear charge
+ model    ::String          ... identifier of the nuclear model: {"Fermi", "Point", "Uniform"}
+ mass     ::Float64         ... atomic mass
+ radius   ::Float64         ... (root-mean square) radius of a uniform or Fermi-distributed nucleus
+ spinI    ::AngularJ64      ... nuclear spin I, must be >= 0
+ mu       ::Float64         ... magnetic dipole moment in Bohr magnetons
+ Q        ::Float64         ... electric qadrupole moment
```

---

`JAC.Nuclear.Model(Z::Real)`  ... constructor just for a given nuclear charge Z, and where a Fermi model is defined       with a = 1.0 and c = 1.0 for the moment. Both, the nuclear spin and moments are all set to zero in this case.

---

`JAC.Nuclear.Model(Z::Real, model::String)`  ... constructor just for a given nuclear charge Z and      model = {"Fermi", "point", "uniform"}, and where further parameters are defined approximately.       Both, the nuclear spin and moments are all set to zero in this case.

---

`JAC.Nuclear.Model(gui::Guint; model::Nuclear.Model=Model(36.0))`  ... constructor that is defined by a graphical user interface.
